# Introduction
In this NoteBook we will clean the datasets of vehicle cost.
# Set up

In [1]:
import pandas as pd
from glob import glob
import os
from pathlib import Path

# Read the data
**Check total number of file in given directory** 

In [2]:
files = glob('../Raw_Data/Vehicle_Cost/*.xlsx')
len(files)

9

**Get the index of 'Annual fuel cost' in excel file**

In [3]:
df_21EV = pd.read_excel('../Raw_Data/Vehicle_Cost/2021 FE Guide-release dates before 11-23-2021-no-sales -11-22-2021 for DOE_Karmapublic.xlsx', 
                      sheet_name = '21')

df_21EV.columns.get_loc('Annual Fuel1 Cost - Conventional Fuel')

44

**Read and calculate average cost of each fuel type by years** 

In [4]:
files = Path('../Raw_Data/Vehicle_Cost').glob('*.xlsx') # get all xlsx files in your dir.

df_output = pd.DataFrame(columns = ['Year','cost_of_gas','cost_of_EV','cost_of_PHEV','cost_of_FCV'])
for file in files:
    # Get annual fuel cost for all types vehicle (different excel sheet)
    df_gas = pd.read_excel(file, sheet_name=0, usecols = [44])
    df_PHEV = pd.read_excel(file, sheet_name=1, usecols = [44]).dropna()[2:] 
    df_EV = pd.read_excel(file, sheet_name=2, usecols = [44]).dropna()[2:]
    df_FCV = pd.read_excel(file, sheet_name=3, usecols = [44]).dropna()[2:]
    
    # Calculate the average cost of the year
    cost_of_gas = df_gas.mean()[0]
    cost_of_PHEV = df_PHEV.mean()[0]
    cost_of_EV = df_EV.mean()[0]
    cost_of_FCV = df_FCV.mean()[0]
    
    # Find the file name which is the year of current df, split with dot, year is the first part.
    year = os.path.basename(file).split(' ')[0]

    # Fill output dataframe with calculated values
    df_output = df_output.append({'Year':year, 
                            'cost_of_gas':cost_of_gas, 
                            'cost_of_EV':cost_of_EV, 
                            'cost_of_PHEV':cost_of_PHEV, 
                            'cost_of_FCV':cost_of_FCV
                           }, ignore_index=True)

df_output = df_output.set_index('Year')
df_output

,cost_of_gas,cost_of_EV,cost_of_PHEV,cost_of_FCV
Year,,,,
2016,2147.732894,664.516129,1311.111111,1475.0
2018,1778.682171,618.750000,1213.235294,1250.0
2021,2095.423729,695.098039,1492.000000,1320.0
2020,2076.817793,646.052632,1445.555556,1350.0
2019,1904.559748,628.571429,1258.333333,1350.0
2015,2532.294165,597.222222,1520.833333,1700.0
2014,2570.443144,620.000000,1315.000000,1400.0
2022,1859.905660,698.125000,1318.604651,1300.0
2017,1794.774920,645.000000,1131.578947,1400.0


**Calculate average cost for all new energy vehicles**

In [5]:
df_output['Avg_NE'] = df_output.iloc[:,1:].mean(axis=1)
df_output.sort_index(inplace=True)
df_output

,cost_of_gas,cost_of_EV,cost_of_PHEV,cost_of_FCV,Avg_NE
Year,,,,,
2014,2570.443144,620.000000,1315.000000,1400.0,1111.666667
2015,2532.294165,597.222222,1520.833333,1700.0,1272.685185
2016,2147.732894,664.516129,1311.111111,1475.0,1150.209080
2017,1794.774920,645.000000,1131.578947,1400.0,1058.859649
2018,1778.682171,618.750000,1213.235294,1250.0,1027.328431
2019,1904.559748,628.571429,1258.333333,1350.0,1078.968254
2020,2076.817793,646.052632,1445.555556,1350.0,1147.202729
2021,2095.423729,695.098039,1492.000000,1320.0,1169.032680
2022,1859.905660,698.125000,1318.604651,1300.0,1105.576550


# Output Cleaned Data Version

In [6]:
path = '../Processed_Data/Save_money.csv'
df_output.to_csv(path)